Data Importation and Exploration

In [ ]:
# installing pyspark
!pip install pyspark

In [24]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number, year, month, dayofmonth, max, corr


In [ ]:

# Start a Spark session
spark = SparkSession.builder.appName('SafaricomStockAnalysis').getOrCreate()

# Load the stock file while inferring the data types
df = spark.read.csv('/content/saf_stock (1).csv', header=True, inferSchema=True)

# Determine the column names
print(df.columns)

# Observations about the schema
df.printSchema()

# Show the first 5 rows
df.show(5)

# Use the describe method to learn about the data frame
df.describe().show()

Data Preparation

In [ ]:
# Format all the data to 2 decimal places
df = df.select([format_number(col, 2).alias(col) if col != 'Date' else col for col in df.columns])

# Create a new data frame with a column called HV Ratio
df2 = df.withColumn('HV Ratio', format_number(df['High']/df['Volume'], 2))
df2.show(5)





Data Analysis


In [ ]:
from pyspark.sql.functions import max

# Find the day with the Peak High Price
max_high = df2.select(max('High')).collect()[0][0]
peak_high_day = df2.filter(df2['High'] == max_high).select('Date').collect()[0][0]
print('The day with the Peak High Price is:', peak_high_day)



In [ ]:
from pyspark.sql.functions import mean

# Find the mean of the Close column
mean_close = df2.select(mean('Close')).collect()[0][0]
print('The mean of the Close column is:', mean_close)



In [ ]:
from pyspark.sql.functions import max, min

# Find the max and min of the Volume column
max_volume = df2.select(max('Volume')).collect()[0][0]
min_volume = df2.select(min('Volume')).collect()[0][0]
print('The maximum volume traded is:', max_volume)
print('The minimum volume traded is:', min_volume)



In [ ]:
from pyspark.sql.functions import count

# Find the number of days when Close was lower than 60 dollars
days_close_below_60 = df2.filter(df2['Close'] < 60).select(count('Date')).collect()[0][0]
print('The number of days when Close was lower than 60 dollars is:', days_close_below_60)



In [ ]:
from pyspark.sql.functions import count, when

# Calculate the percentage of time when High was greater than 80 dollars
total_days = df2.select(count('Date')).collect()[0][0]
high_greater_80_days = df2.filter(df2['High'] > 80).select(count('Date')).collect()[0][0]
high_greater_80_percentage = high_greater_80_days / total_days * 100
print('The percentage of time when High was greater than 80 dollars is:', round(high_greater_80_percentage, 2), '%')



In [ ]:
from pyspark.sql.functions import corr

# Find the Pearson correlation between High and Volume
corr_high_volume = df2.select(corr('High', 'Volume')).collect()[0][0]

if corr_high_volume is not None:
    print("The Pearson correlation between High and Volume is: {:.2f}".format(corr_high_volume))
else:
    print("There are not enough non-null observations to calculate the correlation coefficient.")




In [ ]:
# Compute the max High per year
from pyspark.sql.functions import year, max

# Find the max High per year
max_high_year = df2.groupBy(year('Date')).agg(max('High').alias('Max High')).orderBy('year(Date)')
max_high_year.show()




In [ ]:
# Compute the average Close for each calendar month
from pyspark.sql.functions import month, avg

# Find the average Close for each Calendar Month
avg_close_month = df2.groupBy(month('Date')).agg(avg('Close').alias('Avg Close')).orderBy('month(Date)')
avg_close_month.show()
